In [22]:
import sys

!{sys.executable} -m pip install torch numpy transformers pandas scikit-learn sklearn

In [24]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1IQcFO7AK44MD06auLzL8DeURNyYdoZUf' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1IQcFO7AK44MD06auLzL8DeURNyYdoZUf" -O test_dataset.csv && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1U1CWrwMO8sK_iSRebG1Ueps2At2wmH3z' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1U1CWrwMO8sK_iSRebG1Ueps2At2wmH3z" -O train_dataset.csv && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1I03XYJr5G0lHvRvqbivkDGc_pY1iDPUP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1I03XYJr5G0lHvRvqbivkDGc_pY1iDPUP" -O model_400000.pt && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Xip-j3hszNySLu-FlZFuTiW6Zo-i9uQp' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Xip-j3hszNySLu-FlZFuTiW6Zo-i9uQp" -O sarcasm_detection_model2.pt && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1c0zoLOzsLLpBu1EazWJf8FFppKneJma2' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1c0zoLOzsLLpBu1EazWJf8FFppKneJma2" -O reddit-dataset.csv && rm -rf /tmp/cookies.txt


--2022-03-07 18:28:59--  https://docs.google.com/uc?export=download&confirm=&id=1IQcFO7AK44MD06auLzL8DeURNyYdoZUf
Resolving docs.google.com (docs.google.com)... 172.217.214.139, 172.217.214.101, 172.217.214.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.214.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/es3m65i1i0p49pg0elj3ack1pkvcg36q/1646677725000/15627403865753104419/*/1IQcFO7AK44MD06auLzL8DeURNyYdoZUf?e=download [following]
--2022-03-07 18:28:59--  https://doc-0s-2s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/es3m65i1i0p49pg0elj3ack1pkvcg36q/1646677725000/15627403865753104419/*/1IQcFO7AK44MD06auLzL8DeURNyYdoZUf?e=download
Resolving doc-0s-2s-docs.googleusercontent.com (doc-0s-2s-docs.googleusercontent.com)... 142.251.6.132, 2607:f8b0:4001:c5a::84
Connecting to doc-0s-2s-docs.googleusercontent.com (doc-0s-

In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel
import numpy as np

MODEL = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

roberta = AutoModel.from_pretrained(MODEL).to(device)


Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split


def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


df = pd.read_csv("train_dataset.csv")
X = df["tweet_content"].tolist()
X = [preprocess(x) for x in X]
y = (df["sarcasm_label"] == 'sarcastic').tolist()


In [6]:
from torch.nn.functional import pad

X_encoded = tokenizer(X, return_tensors='pt', padding=True)['input_ids']

In [7]:
X_encoded.shape

torch.Size([2825, 127])

In [8]:
X_train, X_cross, y_train, y_cross = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
y_train = torch.Tensor(y_train).type(torch.long)
y_cross = torch.Tensor(y_cross).type(torch.long)


In [9]:
import torch
from torch import nn

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = roberta
        self.linear1 = nn.Linear(4608, 800)
        self.linear2 = nn.Linear(800, 20)
        self.linear3 = nn.Linear(20, 2)
        self.pooling = nn.MaxPool2d((20, 1))
        self.softmax = nn.Softmax(dim=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.roberta(x).last_hidden_state
        x = self.pooling(x)
        x = x.reshape(x.shape[0], -1)
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout(x)
        x = self.relu(x)
        x = self.linear3(x)
        return x

In [10]:
from torch.utils.data import TensorDataset, DataLoader

dataset_train = TensorDataset(X_train, y_train)  # create your datset
dataloader_train = DataLoader(dataset_train, batch_size=40)  # create your dataloader
dataset_cross = TensorDataset(X_cross, y_cross)  # create your datset
dataloader_cross = DataLoader(dataset_cross, batch_size=40)  # create your dataloader

In [11]:
def train(dataloader, model, loss_fn, optimizer):  #fragment z dokumentacji pytorch
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch % 50 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss :>7f}  [{current:>5d}/{size:>5d}]")
    train_loss /= num_batches
    print(f"Avg train loss: {train_loss:>8f} \n")


In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    true_positive, true_negative, false_negative, false_positive = 0.0001, 0.0001, 0.0001, 0.0001  #zeby dzielenia przez zero nie bylo
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            y_pred = pred.argmax(1)
            correct += (y_pred == y).type(torch.long).sum().item()

            true_positive += torch.logical_and(y_pred == y, y == 1).sum().item()
            true_negative += torch.logical_and(y_pred == y, y == 0).sum().item()
            false_negative += torch.logical_and(y_pred != y, y == 1).sum().item()
            false_positive += torch.logical_and(y_pred != y, y == 0).sum().item()
    recall = true_positive / (true_positive + false_negative)
    precision = true_positive / (true_positive + false_positive)
    specificity = true_negative / (false_positive + true_negative)
    f_score = (2 * precision * recall) / (precision + recall)
    g_mean = (recall * specificity) ** (0.5)

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    print(
        f"\n recall: {recall} \n precision: {precision} \n specificity: {specificity} \n f_score: {f_score} \n g_mean: {g_mean}\n")

In [15]:
model = Model().to(device)
# pretraining na reddicie
#model.load_state_dict(torch.load("model_400000.pt"))
# pretraining na reddicie + trening na isarcasm
model.load_state_dict(torch.load("sarcasm_detection_model2.pt"))

<All keys matched successfully>

In [16]:
loss_fn = nn.CrossEntropyLoss(weight=torch.Tensor([1.0, 1.0])).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
epochs = 15
for t in range(epochs):
    train(dataloader_train, model, loss_fn, optimizer)
    test(dataloader_cross, model, loss_fn)
print("Done!")

loss: 0.041788  [    0/ 2260]
loss: 0.470809  [ 2000/ 2260]
Avg train loss: 0.127231 

Test Error: 
 Accuracy: 79.3%, Avg loss: 0.770665 


 recall: 0.39285739795857627 
 precision: 0.33333367003298986 
 specificity: 0.862785711939413 
 f_score: 0.3606560422818748 
 g_mean: 0.5821956284517735

loss: 0.069235  [    0/ 2260]
loss: 0.259041  [ 2000/ 2260]
Avg train loss: 0.102188 

Test Error: 
 Accuracy: 80.5%, Avg loss: 0.835089 


 recall: 0.3452384637179435 
 precision: 0.3452384637179435 
 specificity: 0.8856547252994905 
 f_score: 0.3452384637179435 
 g_mean: 0.5529575722846495

loss: 0.036276  [    0/ 2260]


KeyboardInterrupt: ignored

In [17]:
test(dataloader_cross, model, loss_fn)

Test Error: 
 Accuracy: 84.2%, Avg loss: 1.006624 


 recall: 0.5119047335601582 
 precision: 0.4725275329065211 
 specificity: 0.9002077338013581 
 f_score: 0.4914285910203634 
 g_mean: 0.6788376831911868



In [18]:
df_test = pd.read_csv("test_dataset.csv")
X_test = df_test["tweet_content"].tolist()
X_test = [preprocess(x) for x in X_test]
y_test = (df_test["sarcasm_label"] == 'sarcastic').tolist()

In [19]:
X_encoded_test = tokenizer(X_test, return_tensors='pt', padding=True)['input_ids']
X_encoded_test = X_encoded_test[:,:127]
y_test = torch.Tensor(y_test).type(torch.long)

In [20]:
dataset_test = TensorDataset(X_encoded_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=40)

In [21]:
test(dataloader_test, model, loss_fn)

Test Error: 
 Accuracy: 83.5%, Avg loss: 0.746108 


 recall: 0.5462184097169585 
 precision: 0.5078124877929878 
 specificity: 0.8934008820978403 
 f_score: 0.52631574685709 
 g_mean: 0.6985642483402728

